# ODIN Kaggle - Train on CIFAR-10 and Test on ImageNet

This notebook is set up to run on Kaggle since they have free GPU hours. It tries to reproduce the results seen in their paper. At a high level, this notebook
- Clones my (Harry's) forked repo of ODIN since their code has some bugs in Python 3 (I'm assuming their syntax was valid in Python 2 or something)
- Downloads their DenseNet-BC trained on CIFAR-10
- Downloads SVHN cropped test set
- Evaluates the model using ODIN and using the baseline

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/harryli0088/odin

Cloning into 'odin'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 41 (delta 10), reused 32 (delta 10), pack-reused 0
Unpacking objects: 100% (41/41), 486.16 KiB | 7.15 MiB/s, done.


In [3]:
# download dense net model
%cd /kaggle/working/odin/models
!wget https://www.dropbox.com/s/wr4kjintq1tmorr/densenet10.pth.tar.gz

/kaggle/working/odin/models
--2022-11-15 04:43:40--  https://www.dropbox.com/s/wr4kjintq1tmorr/densenet10.pth.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/wr4kjintq1tmorr/densenet10.pth.tar.gz [following]
--2022-11-15 04:43:41--  https://www.dropbox.com/s/raw/wr4kjintq1tmorr/densenet10.pth.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucca97043abc7f13d2ae5688f74c.dl.dropboxusercontent.com/cd/0/inline/BwyhDUzJrLN1_A6yfORkrBn2-23lGaPv_i1HnPlDu7-AcgOSGOgMX2ELVSBgBV4WXH4fgdbIzMtOVzUbycuuROz1AILy7Ml99BMFTL-yxPX8eEKvosJmud-zqs_DkhW8Nixc8dxeIJ27T1g1X_fPH0iKZx4XF_gO9YK-aO55ITs8WA/file# [following]
--2022-11-15 04:43:41--  https://ucca97043abc7f13d2ae5688f74c.dl.dropboxusercontent.com/cd/0/inline/BwyhDUzJrLN1_A6yfORkrBn2-

In [4]:
%cd /kaggle/working/odin/models
!tar -xf densenet10.pth.tar.gz
!ls

/kaggle/working/odin/models
densenet10.pth	densenet10.pth.tar.gz


In [5]:
# download SVHN cropped test images
!mkdir /kaggle/working/odin/data
!mkdir /kaggle/working/odin/data/Imagenet
!mkdir /kaggle/working/odin/data/Imagenet/test
%cd /kaggle/working/odin/data
!wget https://www.dropbox.com/s/avgm2u562itwpkl/Imagenet.tar.gz
!tar -xf Imagenet.tar.gz -C ./Imagenet/test
!ls
!ls ./Imagenet/test | head -5

/kaggle/working/odin/data
--2022-11-15 04:43:48--  https://www.dropbox.com/s/avgm2u562itwpkl/Imagenet.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/avgm2u562itwpkl/Imagenet.tar.gz [following]
--2022-11-15 04:43:48--  https://www.dropbox.com/s/raw/avgm2u562itwpkl/Imagenet.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc67408b55b9f49af079cdede5c7.dl.dropboxusercontent.com/cd/0/inline/BwxJGRLEI2lQ1sbN9faM04HBK0IlnCWkVMaqVAp6tFQZG4ioZ3_6r72rbibQAwULxK1ZPjW5BY_PrKH-kqk2YMGiENc9RImXeopVkhZ2zqqlO_FVH_msw5WWM0TvzpFtzea8Sy1MAyrIXOcNIGSRJmOPByzIsZ1QHYgNUMplJ0ys8A/file# [following]
--2022-11-15 04:43:49--  https://uc67408b55b9f49af079cdede5c7.dl.dropboxusercontent.com/cd/0/inline/BwxJGRLEI2lQ1sbN9faM04HBK0IlnCWkVMaqVAp6tFQZG

In [6]:
%cd /kaggle/working/odin/code
# from the ODIN README:
# model: DenseNet-BC, in-distribution: CIFAR-10, out-distribution: TinyImageNet (crop)
# magnitude: 0.0014, temperature 1000, gpu: 0
!python main.py --nn densenet10 --out_dataset Imagenet --magnitude 0.0014 --temperature 1000 --gpu 0

/kaggle/working/odin/code
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the o